In [3]:
!pip install -q -U google-generativeai

import google.generativeai as genai
from kaggle_secrets import UserSecretsClient

# 1. Setup
user_secrets = UserSecretsClient()
api_key = user_secrets.get_secret("GOOGLE_API_KEY")
genai.configure(api_key=api_key)

# 2. List Models
print("SEARCHING FOR AVAILABLE MODELS...")
print("-" * 30)

try:
    available_models = []
    for m in genai.list_models():
        if 'generateContent' in m.supported_generation_methods:
            print(f"✅ FOUND: {m.name}")
            available_models.append(m.name)
            
    if not available_models:
        print("❌ No models found. Your API Key might be invalid or restricted.")
    else:
        print("-" * 30)
        print(f"RECOMMENDED: Please use '{available_models[0]}' in your code.")
        
except Exception as e:
    print(f"Error listing models: {e}")

SEARCHING FOR AVAILABLE MODELS...
------------------------------
✅ FOUND: models/gemini-2.5-pro-preview-03-25
✅ FOUND: models/gemini-2.5-flash
✅ FOUND: models/gemini-2.5-pro-preview-05-06
✅ FOUND: models/gemini-2.5-pro-preview-06-05
✅ FOUND: models/gemini-2.5-pro
✅ FOUND: models/gemini-2.0-flash-exp
✅ FOUND: models/gemini-2.0-flash
✅ FOUND: models/gemini-2.0-flash-001
✅ FOUND: models/gemini-2.0-flash-exp-image-generation
✅ FOUND: models/gemini-2.0-flash-lite-001
✅ FOUND: models/gemini-2.0-flash-lite
✅ FOUND: models/gemini-2.0-flash-lite-preview-02-05
✅ FOUND: models/gemini-2.0-flash-lite-preview
✅ FOUND: models/gemini-2.0-pro-exp
✅ FOUND: models/gemini-2.0-pro-exp-02-05
✅ FOUND: models/gemini-exp-1206
✅ FOUND: models/gemini-2.0-flash-thinking-exp-01-21
✅ FOUND: models/gemini-2.0-flash-thinking-exp
✅ FOUND: models/gemini-2.0-flash-thinking-exp-1219
✅ FOUND: models/gemini-2.5-flash-preview-tts
✅ FOUND: models/gemini-2.5-pro-preview-tts
✅ FOUND: models/learnlm-2.0-flash-experimental
✅ FOU

In [ ]:
# --- 1. FORCE INSTALL & SETUP ---
!pip install -q -U google-generativeai markdown gradio

import google.generativeai as genai
from kaggle_secrets import UserSecretsClient
import json
import os
import time
import gradio as gr

# --- 2. CONFIGURATION ---
MODEL_NAME = "gemini-2.5-flash"

try:
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret("GOOGLE_API_KEY")
    genai.configure(api_key=api_key)
except Exception as e:
    print(f"Error: {e}")

generation_config = {
    "temperature": 0.3,
    "max_output_tokens": 8192,
}
model = genai.GenerativeModel(MODEL_NAME, generation_config=generation_config)

# --- 3. AGENT LOGIC ---
class StudyGenieBot:
    def __init__(self):
        self.reset()
    
    def reset(self):
        self.state = {
            "mode": "WAITING_TOPIC",
            "syllabus": [],
            "topic": "",
            "module_idx": 0,
            "quiz_data": [],
            "quiz_idx": 0,
            "quiz_score": 0,
            "current_lesson_text": ""
        }

    def generate_plan(self, topic):
        prompt = f"""
        Act as a University Dean. Create a 'Zero-to-Hero' syllabus for: '{topic}'.
        Rules: 12 Modules. Beginner to Advanced.
        Return JSON: {{ "modules": ["1. Title", ... "12. Title"] }}
        """
        try:
            response = model.generate_content(prompt)
            text = response.text.strip().replace('```json', '').replace('```', '')
            data = json.loads(text)
            return data['modules']
        except:
            return ["Basics", "Intermediate", "Advanced Concepts"]

    def generate_lesson(self, topic, module):
        prompt = f"""
        Act as a Professor. Write a DEEP-DIVE textbook chapter.
        Topic: {topic}. Chapter: {module}.
        Requirements:
        1. Long, detailed, use headers, analogies, and code examples.
        2. EXTREMELY IMPORTANT: End with a section called '## 🔑 Quick Summary' that lists 5 bullet points for recall.
        """
        response = model.generate_content(prompt)
        return response.text

    def generate_quiz(self, lesson):
        prompt = f"""
        Act as an Examiner. Create 3 Progressive Questions (Easy, Med, Hard).
        Based on: {lesson[:6000]}
        Return JSON list: [{{ "question": "...", "options": ["A)...", "B)..."], "correct_letter": "A" }}]
        Randomize correct answers.
        """
        try:
            response = model.generate_content(prompt)
            text = response.text.strip().replace('```json', '').replace('```', '')
            return json.loads(text)
        except:
            return []

    def respond(self, message):
        mode = self.state["mode"]

        if message.lower() == "/reset":
            self.reset()
            return "♻️ System Reset. What topic would you like to master today?"

        if mode == "WAITING_TOPIC":
            self.state["topic"] = message
            yield f"⚙️ **Architect Agent:** Designing rigorous syllabus for '{message}'... Please wait."
            
            syllabus = self.generate_plan(message)
            self.state["syllabus"] = syllabus
            self.state["mode"] = "PLAN_REVIEW"
            
            plan_str = "\n".join([f"**{m}**" for m in syllabus])
            yield f"### 📘 Course Enrolled: {message}\n\nHere is your Roadmap:\n\n{plan_str}\n\nType **'Start'** to begin Module 1."

        elif mode == "PLAN_REVIEW":
            if "start" in message.lower():
                self.state["mode"] = "LEARNING"
                yield from self.run_teaching_step()
            else:
                yield "Type 'Start' to begin the course, or '/reset' to pick a new topic."

        elif mode == "QUIZ":
            current_q = self.state["quiz_data"][self.state["quiz_idx"]]
            correct = current_q["correct_letter"]
            user_ans = message.strip().upper()
            
            feedback = ""
            if user_ans == correct:
                feedback = f"✅ **Correct!**"
                self.state["quiz_score"] += 1
            else:
                feedback = f"❌ **Incorrect.** The answer was {correct}."
            
            self.state["quiz_idx"] += 1
            
            if self.state["quiz_idx"] < len(self.state["quiz_data"]):
                next_q = self.state["quiz_data"][self.state["quiz_idx"]]
                q_text = self.format_question(next_q, self.state["quiz_idx"]+1)
                yield f"{feedback}\n\n---\n\n{q_text}"
            else:
                score = self.state["quiz_score"]
                total = len(self.state["quiz_data"])
                
                if score >= 2:
                    result_msg = f"{feedback}\n\n### 🎉 Passed! Score: {score}/{total}\nYou have mastered this module."
                    yield result_msg + "\n\nType **'Next'** to proceed to the next module."
                    self.state["mode"] = "NEXT_MODULE_WAIT"
                else:
                    result_msg = f"{feedback}\n\n### ⚠️ Failed. Score: {score}/{total}\nYou must review the material and try again."
                    yield result_msg + "\n\nType **'Retry'** to read the lesson again."
                    self.state["mode"] = "REMEDIAL_WAIT"

        elif mode == "NEXT_MODULE_WAIT":
            if "next" in message.lower():
                self.state["module_idx"] += 1
                if self.state["module_idx"] >= len(self.state["syllabus"]):
                    yield "# 🎓 CONGRATULATIONS! You have officially completed the entire course."
                    self.reset()
                else:
                    self.state["mode"] = "LEARNING"
                    yield from self.run_teaching_step()
            else:
                yield "Type 'Next' to continue."
        
        elif mode == "REMEDIAL_WAIT":
            if "retry" in message.lower():
                self.state["mode"] = "LEARNING"
                yield "🔄 **Reloading Lesson for Review...**"
                yield from self.run_teaching_step(refresh=False)
            else:
                yield "Type 'Retry' to try again."

    def run_teaching_step(self, refresh=True):
        idx = self.state["module_idx"]
        module_title = self.state["syllabus"][idx]
        topic = self.state["topic"]
        
        if refresh:
            yield f"## 📖 Module {idx+1}: {module_title}\n\n**Professor Agent:** Writing textbook chapter... (This may take 10s)"
            lesson = self.generate_lesson(topic, module_title)
            self.state["current_lesson_text"] = lesson
        else:
            lesson = self.state["current_lesson_text"]
        
        yield f"## 📖 Module {idx+1}: {module_title}\n\n{lesson}\n\n---\n\nGenerating Quiz..."
        
        quiz_data = self.generate_quiz(lesson)
        if quiz_data:
            self.state["quiz_data"] = quiz_data
            self.state["quiz_idx"] = 0
            self.state["quiz_score"] = 0
            self.state["mode"] = "QUIZ"
            
            first_q = quiz_data[0]
            q_text = self.format_question(first_q, 1)
            yield f"## 📖 Module {idx+1}: {module_title}\n\n{lesson}\n\n---\n\n### 📝 Pop Quiz\n{q_text}"
        else:
            self.state["mode"] = "NEXT_MODULE_WAIT"
            yield f"{lesson}\n\n(No quiz generated). Type 'Next' to continue."

    def format_question(self, q_obj, num):
        opts = "\n".join(q_obj['options'])
        return f"**Q{num}: {q_obj['question']}**\n{opts}\n\n*(Type A, B, C, or D)*"

# --- 4. UI LAUNCHER (UNIVERSAL LIST FORMAT) ---
bot = StudyGenieBot()

# Initial Greeting
initial_history = [
    [None, "👋 **Welcome to StudyGenie Pro!**\n\nI am your Personal Learning Concierge. I will design a full 'Zero-to-Hero' course for you, teach you everyday, and test your knowledge.\n\n**Type a topic below to begin (e.g., 'Python', 'Digital Marketing').**"]
]

with gr.Blocks() as demo:
    gr.Markdown("# 🧞‍♂️ StudyGenie Pro")
    
    # Using Standard List format (Works on ALL Gradio versions)
    chatbot = gr.Chatbot(value=initial_history, height=600)
    
    with gr.Row():
        msg = gr.Textbox(
            scale=4,
            show_label=False, 
            placeholder="Type your topic here (e.g. 'Advanced Python') and press Enter..."
        )
        clear = gr.Button("Reset", scale=1)

    # Handler for List of Lists [[user, bot]]
    def bot_response(user_message, history):
        if user_message.strip() == "":
            return "", history
            
        # Append User Message
        history = history + [[user_message, None]]
        
        # Get Generator
        response_generator = bot.respond(user_message)
        
        # Stream Response
        for partial_response in response_generator:
            history[-1][1] = partial_response
            yield "", history

    msg.submit(bot_response, [msg, chatbot], [msg, chatbot], queue=False)
    clear.click(lambda: None, None, chatbot, queue=False)

demo.queue().launch(share=True, debug=True)